In [45]:
import csv
import cv2
import numpy as np
from enum import Enum

correction = 0.3

lines = []

with open('../cardata/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)


class Augmentation(Enum):
    Original = 0
    Left = 1
    Right = 2
    Reverse_Original = 3
    Reverse_Left = 4
    Reverse_Right = 5
    
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(lines, test_size=0.2)

import cv2
import numpy as np
import sklearn
import itertools



def load_image(source_path):
    #filename = source_path.split('/')[-1]
    #current_path = '../data/IMG/' + filename 
    return cv2.imread(source_path)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    augmented_samples = list(itertools.product(samples, [aug for aug in Augmentation]))
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(augmented_samples)

        for offset in range(0, num_samples, batch_size):
            batch_samples = augmented_samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:

                steering = float(batch_sample[0][3])
                if batch_sample[1].value % 3 == 0:
                    image = load_image(batch_sample[0][0])
                if batch_sample[1].value % 3 == 1:
                    image = load_image(batch_sample[0][1])
                    steering = steering + correction
                if batch_sample[1].value % 3 == 2:
                    image = load_image(batch_sample[0][2])
                    steering = steering - correction
                if batch_sample[1].value > 2:
                    image = np.fliplr(image)
                    steering = steering * -1.0
                
                images.append(image)
                angles.append(steering)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)        



In [34]:
def plot_history(history_object):
    ### print the keys contained in the history object
    print(history_object.history.keys())

    ### plot the training and validation loss for each epoch
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.show()

In [55]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras.models import Model

# this could also be the output a different Keras model or layer
input_tensor = Input(shape=(160, 320, 3))  # this assumes K.image_data_format() == 'channels_last'

base_model = DenseNet121(input_tensor=input_tensor, weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
prediction = Dense(1)(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=prediction)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='mse', optimizer='adam')


train_generator = generator(traLosin_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)


history_object = model.fit_generator(
    train_generator,
    samples_per_epoch=len(train_samples)*len(Augmentation), 
    validation_data=validation_generator,
    validation_steps = len(validation_samples)*len(Augmentation), 
    epochs=5, 
    verbose=1
)


plot_history(history_object)

model.save('model_tf.h5')


/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=5, validation_data=<generator..., verbose=1, steps_per_epoch=9312, validation_steps=2334)`


Epoch 1/5
 544/9312 [>.............................] - ETA: 12:29:58 - loss: 0.0758

KeyboardInterrupt: 

In [ ]:
print(model.summary())

In [ ]:

train_generator = generator(traLosin_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)


history_object = model.fit_generator(
    train_generator,
    samples_per_epoch=len(train_samples)*len(Augmentation), 
    validation_data=validation_generator,
    validation_steps = len(validation_samples)*len(Augmentation), 
    epochs=5, 
    verbose=1
)


plot_history(history_object)

model.save('model_tf.h5')

In [43]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


0 input_5
1 conv2d_377
2 batch_normalization_377
3 activation_377
4 conv2d_378
5 batch_normalization_378
6 activation_378
7 conv2d_379
8 batch_normalization_379
9 activation_379
10 max_pooling2d_17
11 conv2d_380
12 batch_normalization_380
13 activation_380
14 conv2d_381
15 batch_normalization_381
16 activation_381
17 max_pooling2d_18
18 conv2d_385
19 batch_normalization_385
20 activation_385
21 conv2d_383
22 conv2d_386
23 batch_normalization_383
24 batch_normalization_386
25 activation_383
26 activation_386
27 average_pooling2d_37
28 conv2d_382
29 conv2d_384
30 conv2d_387
31 conv2d_388
32 batch_normalization_382
33 batch_normalization_384
34 batch_normalization_387
35 batch_normalization_388
36 activation_382
37 activation_384
38 activation_387
39 activation_388
40 mixed0
41 conv2d_392
42 batch_normalization_392
43 activation_392
44 conv2d_390
45 conv2d_393
46 batch_normalization_390
47 batch_normalization_393
48 activation_390
49 activation_393
50 average_pooling2d_38
51 conv2d_389
52